In [20]:
import cv2
import numpy as np
import json
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Đường dẫn tới thư mục dữ liệu
DATA_PATH = r'C:\Users\USER\Desktop\golf_swing\golf_dataset'
VIDEO_PATH = os.path.join(DATA_PATH, 'videos')
ANNOTATION_PATH = os.path.join(DATA_PATH, 'annotations')

# Hàm trích xuất tổng số frame từ video
def get_total_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f'Error: Cannot open video file {video_path}')
        return 0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    return total_frames

# Hàm trích xuất frame từ video và gắn nhãn phase
def extract_frames_and_labels(video_file, annotation_file):
    frames = []
    labels = []
    video_path = os.path.join(VIDEO_PATH, video_file)
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f'Error: Cannot open video file {video_path}')
        return np.array([]), np.array([])
    
    try:
        with open(annotation_file, 'r') as f:
            annotations = json.load(f)
    except FileNotFoundError:
        print(f'Error: Annotation file {annotation_file} not found')
        return np.array([]), np.array([])
    except json.JSONDecodeError:
        print(f'Error: Invalid JSON format in {annotation_file}')
        return np.array([]), np.array([])
    
    phase_to_label = {'setup': 0, 'backswing': 1, 'downswing': 2, 'follow-through': 3}
    frame_count = 0
    labeled_frames = 0
    total_frames = get_total_frames(video_path)
    
    while cap.isOpened() and frame_count < total_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (128, 128))
        frames.append(frame)
        
        label = None
        for phase, info in annotations.items():
            try:
                start, end = info['start_frame'], info['end_frame']
                if start <= frame_count <= end:
                    label = phase_to_label[phase]
                    labeled_frames += 1
                    break
            except KeyError:
                print(f'Error: Invalid phase data in {annotation_file} for phase {phase}')
                continue
        if label is not None:
            labels.append(label)
        else:
            print(f'Warning: Frame {frame_count} in {video_file} has no phase label')
        frame_count += 1
    
    cap.release()
    print(f'Info: {labeled_frames}/{total_frames} frames labeled in {video_file}')
    if labeled_frames == 0:
        print(f'Error: No frames labeled in {video_file}')
    return np.array(frames), np.array(labels)

# Load toàn bộ dữ liệu
def load_dataset():
    all_frames = []
    all_labels = []
    
    if not os.path.exists(VIDEO_PATH):
        print(f'Error: Directory {VIDEO_PATH} does not exist')
        return np.array([]), np.array([])
    
    video_files = [f for f in os.listdir(VIDEO_PATH) if f.endswith('.mp4')]
    if not video_files:
        print(f'Error: No .mp4 files found in {VIDEO_PATH}')
        return np.array([]), np.array([])
    
    for video_file in video_files:
        video_path = os.path.join(VIDEO_PATH, video_file)
        annotation_file = os.path.join(ANNOTATION_PATH, f'{video_file[:-4]}_error.json')
        if not os.path.exists(annotation_file):
            print(f'Error: No annotation file for {video_file}')
            continue
        frames, labels = extract_frames_and_labels(video_path, annotation_file)
        if len(frames) > 0 and len(labels) > 0:
            all_frames.append(frames)
            all_labels.append(labels)
        else:
            print(f'Warning: No valid frames or labels for {video_file}')
    
    if not all_frames:
        print('Error: No valid data loaded')
        return np.array([]), np.array([])
    
    return np.concatenate(all_frames), np.concatenate(all_labels)

def main():
    # Chuẩn bị dữ liệu
    frames, labels = load_dataset()
    if len(frames) == 0:
        print('Exiting due to no valid data')
        return
    
    frames = frames / 255.0  # Chuẩn hóa giá trị pixel
    labels = to_categorical(labels, num_classes=4)  # One-hot encoding cho 4 phase

    # Chia tập train/test
    X_train, X_test, y_train, y_test = train_test_split(frames, labels, test_size=0.2, random_state=42)

    # Data augmentation
    datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True
    )
    datagen.fit(X_train)

    # Xây dựng mô hình CNN
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(4, activation='softmax')  # 4 phase
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

    # Huấn luyện mô hình với augmentation
    model.fit(datagen.flow(X_train, y_train, batch_size=32), 
              epochs=15, 
              validation_data=(X_test, y_test))

    # Lưu mô hình
    model.save('phase_classifier.h5')

    # Đánh giá mô hình
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f'Test accuracy: {accuracy:.4f}')

if __name__ == '__main__':
    main()

Info: 75/75 frames labeled in C:\Users\USER\Desktop\golf_swing\golf_dataset\videos\video_001.mp4
Info: 54/54 frames labeled in C:\Users\USER\Desktop\golf_swing\golf_dataset\videos\video_002.mp4
Info: 79/79 frames labeled in C:\Users\USER\Desktop\golf_swing\golf_dataset\videos\video_003.mp4
Info: 55/55 frames labeled in C:\Users\USER\Desktop\golf_swing\golf_dataset\videos\video_004.mp4
Info: 54/54 frames labeled in C:\Users\USER\Desktop\golf_swing\golf_dataset\videos\video_005.mp4


C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 253ms/step - accuracy: 0.2146 - loss: 3.6129 - val_accuracy: 0.2812 - val_loss: 1.3805
Epoch 2/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 226ms/step - accuracy: 0.3523 - loss: 2.8733 - val_accuracy: 0.3125 - val_loss: 1.3786
Epoch 3/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 233ms/step - accuracy: 0.2841 - loss: 2.2972 - val_accuracy: 0.2031 - val_loss: 1.4062
Epoch 4/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 218ms/step - accuracy: 0.3186 - loss: 1.8282 - val_accuracy: 0.2031 - val_loss: 1.4930
Epoch 5/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 219ms/step - accuracy: 0.2888 - loss: 1.6445 - val_accuracy: 0.2031 - val_loss: 1.7216
Epoch 6/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 337ms/step - accuracy: 0.3576 - loss: 1.4387 - val_accuracy: 0.2031 - val_loss: 1.9792
Epoch 7/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 348ms/step - accuracy: 0.4106 - loss: 1.3526 - val_accuracy: 0.2031 - val_loss: 2.3102
Epoch 8/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 354ms/step - accuracy: 0.3280 - loss: 1.3712 - val_accuracy: 0.2031 - val_loss:

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.1875 - loss: 4.8929
Test accuracy: 0.2031
